# Extracting insights using SQL

This notebook contains SQL queries for extracting the following insights:
- Q4.  total and average admissions for each month over the last two years
- Q5.  distribution of TotalCharges by PrincipalDiagnosis and Sex

## Q4. Total + Ave admissions by month x year for last 2 years

In [25]:
import pandas as pd

# Ingest raw data
input_filepath = "../data/Data Insights - Synthetic Dataset.csv"
input_df = pd.read_csv(input_filepath)

In [26]:
import pandas as pd
import pandasql as psql

# Convert 'AdmissionDate' to datetime
input_df['AdmissionDate'] = pd.to_datetime(input_df['AdmissionDate'], format='%d/%m/%Y')

# Extract Month and Year columns for SQL
input_df['Month'] = input_df['AdmissionDate'].dt.strftime('%b')
input_df['Year'] = input_df['AdmissionDate'].dt.year

# Define the SQL query
query = """
WITH MonthlyAdmissions AS (
    SELECT 
        Month,
        Year,
        COUNT(*) AS TotalAdmissions
    FROM input_df
    WHERE Year IN (2022, 2023, 2024)
    GROUP BY Year, Month
),
YearlyAdmissions AS (
    SELECT 
        Month,
        SUM(CASE WHEN Year = 2022 THEN TotalAdmissions ELSE 0 END) AS Year2022,
        SUM(CASE WHEN Year = 2023 THEN TotalAdmissions ELSE 0 END) AS Year2023,
        SUM(CASE WHEN Year = 2024 THEN TotalAdmissions ELSE 0 END) AS Year2024
    FROM MonthlyAdmissions
    GROUP BY Month
),
FinalAdmissions AS (
    SELECT 
        Month,
        Year2022,
        Year2023,
        Year2024,
        (Year2022 + Year2023 + Year2024) AS Total_admissions,
        ROUND((Year2022 + Year2023 + Year2024) / 2,1) AS Average_admissions
    FROM YearlyAdmissions
)
SELECT 
    Month,
    COALESCE(Year2022, 0) AS `2022`,
    COALESCE(Year2023, 0) AS `2023`,
    COALESCE(Year2024, 0) AS `2024`,
    COALESCE(Total_admissions, 0) AS `Total admissions`,
    COALESCE(Average_admissions, 0) AS `Average admissions`
FROM FinalAdmissions
ORDER BY CASE
    WHEN Month = 'Jan' THEN 1
    WHEN Month = 'Feb' THEN 2
    WHEN Month = 'Mar' THEN 3
    WHEN Month = 'Apr' THEN 4
    WHEN Month = 'May' THEN 5
    WHEN Month = 'Jun' THEN 6
    WHEN Month = 'Jul' THEN 7
    WHEN Month = 'Aug' THEN 8
    WHEN Month = 'Sep' THEN 9
    WHEN Month = 'Oct' THEN 10
    WHEN Month = 'Nov' THEN 11
    WHEN Month = 'Dec' THEN 12
    ELSE 13
END;
"""

# Execute the query
result_df = psql.sqldf(query, locals())

# Display the results
print(result_df)


   Month  2022  2023  2024  Total admissions  Average admissions
0    Jan     0  1252  1260              2512              1256.0
1    Feb     0  1166  1171              2337              1168.0
2    Mar     0  1259  1287              2546              1273.0
3    Apr     0  1252  1266              2518              1259.0
4    May     0  1272  1344              2616              1308.0
5    Jun     0  1267  1286              2553              1276.0
6    Jul     0  1287  1265              2552              1276.0
7    Aug  1215  1233     0              2448              1224.0
8    Sep  1236  1189     0              2425              1212.0
9    Oct  1238  1322     0              2560              1280.0
10   Nov  1245  1187     0              2432              1216.0
11   Dec  1244  1257     0              2501              1250.0


# Q5.  distribution of TotalCharges by PrincipalDiagnosis and Sex

NOTE: In the SQL query below, it would be best to use the PERCENTILE_CONT function, however this is not supported in Pandasql so PERCENTILERANK has been used instead.
I also output the total_sample_size in each slice in the final result. Given the small dataset we find that the sample sizes are very small (often only 1 patient), and hence the percentile distributions aren't very descriptive.  A larger dataset would solve this issue.

In [39]:
import pandas as pd
import pandasql as psql

# Convert charge columns to numeric types
charge_columns = [
    'AccommodationCharge', 'CCU_Charges', 'ICU_Charge', 
    'TheatreCharge', 'PharmacyCharge', 'ProsthesisCharge', 
    'OtherCharges', 'BundledCharges'
]
for col in charge_columns:
    input_df[col] = pd.to_numeric(input_df[col], errors='coerce')

# Define the SQL query using a CTE to calculate TotalCharges
query = """
WITH TotalChargesCTE AS (
    SELECT *,
        COALESCE(AccommodationCharge, 0) + 
        COALESCE(CCU_Charges, 0) + 
        COALESCE(ICU_Charge, 0) + 
        COALESCE(TheatreCharge, 0) + 
        COALESCE(PharmacyCharge, 0) + 
        COALESCE(ProsthesisCharge, 0) + 
        COALESCE(OtherCharges, 0) + 
        COALESCE(BundledCharges, 0) AS TotalCharges
    FROM input_df
),
PercentilesCTE AS (
    SELECT 
        PrincipalDiagnosis,
        Sex,
        TotalCharges,
        PERCENT_RANK() OVER (PARTITION BY PrincipalDiagnosis, Sex ORDER BY TotalCharges) AS PercentileRank
    FROM TotalChargesCTE
)
SELECT 
    PrincipalDiagnosis,
    Sex,
    MAX(CASE WHEN PercentileRank <= 0.25 THEN TotalCharges ELSE NULL END) AS Percentile_25,
    MAX(CASE WHEN PercentileRank <= 0.50 THEN TotalCharges ELSE NULL END) AS Percentile_50,
    MAX(CASE WHEN PercentileRank <= 0.75 THEN TotalCharges ELSE NULL END) AS Percentile_75,
    COUNT(*) AS total_sample_size
FROM PercentilesCTE
GROUP BY PrincipalDiagnosis, Sex
ORDER BY PrincipalDiagnosis, Sex;
"""

# Execute the query
result_df = psql.sqldf(query, locals())

# Display the results
print(result_df)


      PrincipalDiagnosis Sex  Percentile_25  Percentile_50  Percentile_75  \
0                  A00.1   F   1.582878e+03   1.582878e+03   1.582878e+03   
1                  A00.2   F   9.292856e+97   9.292856e+97   9.292856e+97   
2                  A00.2   M   3.767676e+71   3.767676e+71   3.767676e+71   
3                  A00.4   F   5.506981e+02   5.506981e+02   5.506981e+02   
4                  A00.4   M   3.011643e+03   3.011643e+03   3.011643e+03   
...                  ...  ..            ...            ...            ...   
22852              Z99.3   F  1.845434e+115  1.845434e+115  1.845434e+115   
22853              Z99.3   M   5.803964e+01   5.803964e+01   5.803964e+01   
22854              Z99.7   F   1.522418e+01   1.522418e+01   1.522418e+01   
22855              Z99.8   F   4.899061e+70   4.899061e+70   4.899061e+70   
22856              Z99.9   F   5.356767e+01   5.356767e+01   5.356767e+01   

       total_sample_size  
0                      1  
1                    